# Recommendations: Part 1

In this notebook you will learn how to make recommendations using Neo4j. 

Execute the code to import the libraries (remember to unset Reset all runtimes before running):

In [2]:
!pip install py2neo

     |████████████████████████████████| 184kB 5.8MB/s 
     |████████████████████████████████| 348kB 39.2MB/s 
     |████████████████████████████████| 2.6MB 45.1MB/s 
     |████████████████████████████████| 153kB 42.9MB/s 
     |████████████████████████████████| 204kB 46.4MB/s 
  Created wheel for neotime: filename=neotime-1.7.4-cp36-none-any.whl size=20543 sha256=05192d8eaed8bb8ea6dadb8c69a23b2ef5f32314d60ee7c3f9a34fdf3048bcb4
  Stored in directory: /root/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neotime
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.10 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18


In [3]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

Next, create a connection to your Neo4j Sandbox, just as you did previously when you set up your environment. 

<div align="left">
    <img src="https://github.com/neo4j-contrib/training-v2/blob/master/Courses/DataScience/notebooks/images/sandbox-citations.png?raw=1" alt="Citation Sandbox"/>
</div>

Update the cell below to use the IP Address, Bolt Port, and Password, as you did previously.

In [4]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 
 
graph = Graph("bolt://52.201.185.131:33570", auth=("neo4j", "raises-hardcopy-flashlights"))

##  Finding popular authors

Since we're going to make collaborator suggestions find authors who have written the most articles so that we have some data to work with.

In [5]:
popular_authors_query = """
MATCH (author:Author)
RETURN author.name, size((author)<-[:AUTHOR]-()) AS articlesPublished
ORDER BY articlesPublished DESC
LIMIT 10
"""

graph.run(popular_authors_query).to_data_frame()

,author.name,articlesPublished
0,Peter G. Neumann,89
1,Peter J. Denning,80
2,Moshe Y. Vardi,72
3,Pamela Samuelson,71
4,Bart Preneel,65
5,Vinton G. Cerf,56
6,Barry W. Boehm,53
7,Mark Guzdial,49
8,Edwin R. Hancock,47
9,Josef Kittler,46


Pick one of these authors...

In [6]:
author_name = "Peter G. Neumann"

Retrieve the articles they've published and how many citations they've received:

In [7]:
author_articles_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)
RETURN article.title AS article, article.year AS year, size((article)<-[:CITED]-()) AS citations
ORDER BY citations DESC
LIMIT 20
"""

graph.run(author_articles_query,  {"authorName": author_name}).to_data_frame()

,article,year,citations
0,"The foresight saga, redux",2012,2
1,Security by obscurity,2003,2
2,Risks of automation: a cautionary total-system perspective of our cyberfuture,2016,1
3,Crypto policy perspectives,1994,1
4,Risks of National Identity Cards,2001,1
5,"Computers, ethics, and values",1991,1
6,Are dependable systems feasible,1993,1
7,Information system security redux,2003,1
8,The foresight saga,2006,1
9,Robust open-source software,1999,1


Find the author's collaborators:

In [8]:
collaborations_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor)
RETURN coauthor.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,Lauren Weinstein,3
1,Whitfield Diffie,3
2,Susan Landau,3
3,Steven Michael Bellovin,2
4,Matt Blaze,2
5,Rebecca T. Mercuri,2
6,Alfred Z. Spector,1
7,Seymour E. Goodman,1
8,David Lorge Parnas,1
9,Douglas Miller,1


How would you suggest some future collaborators for this author? One way is by looking at the collaborators of their collaborators!

In [9]:
collaborations_query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
RETURN coc.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,John Ioannidis,10
1,Scott Bradner,9
2,Angelos D. Keromytis,8
3,John Kelsey,7
4,Virgil D. Gligor,5
5,David Wagner,4
6,Peter Wolcott,4
7,Ran Canetti,4
8,Gerald Jay Sussman,4
9,David K. Gifford,4


Each of these people have collaborated with someone that Peter has worked with before, so they might be able to do an introduction.

## Exercise

1. Can you find the top 20 suggested collaborators for 'Brian Fitzgerald' instead of 'Peter G. Neumann'?
2. How many of these potential collaborators have collaborated with Brian's collaborators more than 3 times?

Keep the results of this exercise handy as they may be useful for the quiz at the end of this module.